In [17]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
import pprint

In [72]:
person_list = []

def person_df():
    df = pd.DataFrame(person_list,
            columns=['Name', 'ID']+[f'Q{i+1}' for i in range(num_questions)],
                            )
    df.set_index(['Name', 'ID'], inplace=True)
    return df

In [70]:
num_questions = 3

In [73]:
name = widgets.Text(
 placeholder='Name',
 description='Name:'
 )

num = widgets.IntText(
 placeholder='ID Num',
 description='ID Num:'
 )

scores = widgets.Box([
    widgets.IntText(
        placeholder=f'Score {i}',
        description='')
    for i in range(num_questions)]
)


button = widgets.Button(
    description='Add person',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add person',
    icon='plus' # (FontAwesome names without the `fa-` prefix)
)

output = widgets.Output()

def add_to_list(*args):
    person_list.append([name.value, num.value] + [child.value for child in scores.children])
    with output:
        output.clear_output()
        display(person_df())
        

button.on_click(add_to_list)

display(name)
display(num)
display(scores)
display(button)
display(output)

Text(value='', description='Name:', placeholder='Name')

IntText(value=0, description='ID Num:')

Box(children=(IntText(value=0), IntText(value=0), IntText(value=0)))

Button(button_style='success', description='Add person', icon='plus', style=ButtonStyle(), tooltip='Add person…

Output()

In [19]:
def biggest_difference(row1, row2):
    return (row1-row2).abs().max()

In [80]:
df = person_df()

In [30]:
def build_distance_matrix(response_df, metric=biggest_difference):

    distance_matrix = pd.DataFrame(index=response_df.index,
                                   columns=response_df.index)

    for idx, row in response_df.iterrows():
        for idy, col in response_df.iterrows():        
            distance_matrix.loc[idx, idy] = metric(row, col)

    return distance_matrix

In [121]:
build_distance_matrix(person_df())

,Name,Bertie,Jeremy,Μολλυ,"Kar,",Adam
,ID,8,3,9,10,5
Name,ID,,,,,
Bertie,8,0,8,1,7,4
Jeremy,3,8,0,7,3,7
Μολλυ,9,1,7,0,6,4
"Kar,",10,7,3,6,0,6
Adam,5,4,7,4,6,0


In [122]:
dm = build_distance_matrix(person_df())

In [148]:
def random_pairs(distance_matrix):
    shuffled_index = dm.sample(frac=1).index
    pairs = np.array_split(shuffled_index, np.ceil(len(shuffled_index)/2))
    return pairs

In [151]:
random_pairs(dm)

[Index([('Bertie', 8), ('Adam', 5)], dtype='object'),
 Index([('Kar,', 10), ('Μολλυ', 9)], dtype='object'),
 Index([('Jeremy', 3)], dtype='object')]

In [184]:
def score_pair(pair, matrix):
    return matrix.loc[pair[0], pair[1]]

In [185]:
def total_pair_score(pairs, dm):
    return sum([score_pair(p, dm) for p in pairs if len(p)>1])

In [188]:
def min_pair_score(pairs, dm):
    return min([score_pair(p, dm) for p in pairs if len(p)>1])

In [192]:
pairs = random_pairs(dm)
pairs

[Index([('Bertie', 8), ('Adam', 5)], dtype='object'),
 Index([('Μολλυ', 9), ('Jeremy', 3)], dtype='object'),
 Index([('Kar,', 10)], dtype='object')]

In [212]:
type(pairs[0])

pandas.core.indexes.base.Index

In [193]:
total_pair_score(pairs, dm)

11

In [194]:
min_pair_score(pairs, dm)

4

In [213]:
def pair_to_tuple(pair: pd.Index):
    return tuple(pair.values.tolist())

In [211]:
shuffles

{(('Bertie', 8), ('Adam', 5)): 1}

In [218]:
shuffles = dict()

for i in range(5):
    pairs = random_pairs(dm)
    
    shuffles[tuple(pair_to_tuple(p) for p in pairs)] = {'total': total_pair_score(pairs, dm),
                                                   'min': min_pair_score(pairs, dm)
                                                  }
    
shuffles

{((('Jeremy', 3), ('Kar,', 10)),
  (('Adam', 5), ('Bertie', 8)),
  (('Μολλυ', 9),)): {'total': 7, 'min': 3},
 ((('Jeremy', 3), ('Kar,', 10)),
  (('Bertie', 8), ('Adam', 5)),
  (('Μολλυ', 9),)): {'total': 7, 'min': 3},
 ((('Bertie', 8), ('Kar,', 10)),
  (('Jeremy', 3), ('Adam', 5)),
  (('Μολλυ', 9),)): {'total': 14, 'min': 7},
 ((('Bertie', 8), ('Kar,', 10)),
  (('Adam', 5), ('Jeremy', 3)),
  (('Μολλυ', 9),)): {'total': 14, 'min': 7},
 ((('Μολλυ', 9), ('Jeremy', 3)),
  (('Kar,', 10), ('Bertie', 8)),
  (('Adam', 5),)): {'total': 14, 'min': 7}}

In [36]:
def max_pair(distance_matrix):
    player1 = distance_matrix.max().idxmax()
    player2 = distance_matrix.idxmax()[player1]
    
    return(player1, player2)

In [220]:
dm.idxmax()

Name    ID
Bertie  8     (Jeremy, 3)
Jeremy  3     (Bertie, 8)
Μολλυ   9     (Jeremy, 3)
Kar,    10    (Bertie, 8)
Adam    5     (Jeremy, 3)
dtype: object

In [222]:
dm.max().idxmax()

('Bertie', 8)

In [232]:
dm.columns.size

5

In [230]:
max_pair(distance_matrix)

(('Bertie', 8), ('Jeremy', 3))

In [33]:
def greedy_sort(distance_matrix):
    
    sorted_pairs = []
    
    dm = distance_matrix.copy()
    
    while dm.columns.size > 1:
        a, b = max_pair(dm)

        sorted_pairs.append((a, b))

        dm.drop([a, b], inplace=True, axis='columns')
        dm.drop([a, b], inplace=True, axis='index')
        
    if dm.columns.size > 0:
        sorted_pairs.append(dm.max().idxmax())
        
    return sorted_pairs

In [264]:
print('\n'.join(str(p) for p in greedy_sort(dm)))

(('Bertie', 8), ('Jeremy', 3))
(('Μολλυ', 9), ('Kar,', 10))
('Adam', 5)


In [240]:
dm

,Name,Bertie,Jeremy,Μολλυ,"Kar,",Adam
,ID,8,3,9,10,5
Name,ID,,,,,
Bertie,8,0,8,1,7,4
Jeremy,3,8,0,7,3,7
Μολλυ,9,1,7,0,6,4
"Kar,",10,7,3,6,0,6
Adam,5,4,7,4,6,0


In [5]:
import pandas as pd

sheet_id = "1zCjFOHdktWWpfdqu5MRBTiZAhYZVUSH836XVVYrUsFQ"
sheet_name = "Form responses 1"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"

In [13]:
#pd.read_csv(url,on_bad_lines='skip')
# Authentication problem

In [12]:
#import sys
#!{sys.executable} -m pip install pygsheets

In [14]:
#import pygsheets

#client = pygsheets.authorize()

# Open the spreadsheet and the first sheet.
#sh = client.open('spreadsheet-title')
#wks = sh.sheet1

# needs 'client_secret.json'

In [15]:
csv_filename = '/Users/sam/Downloads/Double Crux Questionnaire (Responses) - Form responses 1.csv'

In [23]:
pd.read_csv(csv_filename, index_col='Name')

,Timestamp,"""If my country asked me to fight a war, I would.""","""It's sometimes necessary to censor books.""","""It's OK to burn a country's flag."""
Name,,,,
Peace,12/07/2022 14:02:24,1,4,4
War,12/07/2022 14:07:21,7,4,4
Censor,12/07/2022 14:08:26,4,7,4
Test,12/07/2022 14:13:47,4,4,4
Free speech,13/07/2022 14:08:26,4,1,4


In [64]:
dm = build_distance_matrix(
        pd.read_csv(csv_filename, index_col='Name'
               ).drop('Timestamp', axis='columns'
                     )
).apply(pd.to_numeric)
dm

Name,Peace,War,Censor,Test,Free speech
Name,,,,,
Peace,0,6,3,3,3
War,6,0,3,3,3
Censor,3,3,0,3,6
Test,3,3,3,0,3
Free speech,3,3,6,3,0


In [65]:
greedy_sort(dm)

[('Peace', 'War'), ('Censor', 'Free speech'), 'Test']